$10,000$ MC replications ($s=10000$), $n=200, 400, 1000$,  
$E[Y|X] = \theta_0^2 X + \theta_0 X^2$, $\theta_0 = 5/4$, $X \sim N(\mu,1)$, $\mu=1$ in (a), $\mu=-1$ in (b).  
$\text{var}(Y|X)$ is constant.

Moments: with $\rho(X,\theta) = 2\theta X + X^2$,  
$(a): \mathbb{E}[\rho(X,\theta_0)(Y-\theta^2 X - \theta X^2)]$  
$(b): \mathbb{E}[\rho(X,\theta)(Y-\theta^2 X - \theta X^2)]$

Do GMM with initial values $-4, -2, 0, 2$ randomly.

In [1]:
#!pip install tikzplotlib

In [2]:
import numpy as np
from numba import njit,jit
import time

import pandas as pd
from scipy import optimize
import sys
import matplotlib.pyplot as plt
import tikzplotlib

In [9]:
# DGP
def my_dgp(n, mu):
    """
    n = sample size, mu = 1 or -1.
    """
    theta_0 = 5/4
    X = np.random.normal(size=n, loc=mu, scale=1)
    if len(X.shape) == 1:
        X = X.reshape(n,1)
    U = np.random.normal(size=n, loc=0, scale=1) * X
    Y = (theta_0**2)*X + theta_0 * (X**2)
    
    return Y, X

@jit
def my_dgp2(n, mu):
    """
    n = sample size, mu = 1 or -1.
    """
    theta_0 = 5/4
    X = np.random.normal(size=n, loc=mu, scale=1)
    if len(X.shape) == 1:
        X = X.reshape(n,1)
    U = np.random.normal(size=n, loc=0, scale=1) * X
    Y = (theta_0**2)*X + theta_0 * (X**2)
    
    return Y, X


# GMM
def my_gmm(Y,X, moment, init):
    """
    moment = 'a' or 'b'
    """
    n,_ = X.shape
    
    # moment
    rho = lambda X,theta: 2*theta*X + X**2
    moment_a = lambda Y,X,theta: rho(X, 5/4) * (Y - (theta**2)*X - theta*(X**2))
    moment_b = lambda Y,X,theta: rho(X, theta) * (Y - (theta**2)*X - theta*(X**2))
    if moment == 'a':
        mom=moment_a
    elif moment == 'b':
        mom=moment_b
    else:
        print("moment should be 'a' or 'b'")
                                                  
    # objective function
    def obj(theta):
        g = lambda theta: mom(Y,X, theta).mean()
        J = lambda theta: n*(g(theta)**2)
        
        theta1 = optimize.fmin(J, init, disp=False)
        e = mom(Y,X,theta1)
        omega = (e-e.mean()).T@(e-e.mean())/n  # centered
        
        obj = n * g(theta)**2 / omega
        return obj
    coef = optimize.fmin(obj, init, disp=False)
    return coef[0]

@njit
def _rho(X,theta):
    return 2*theta*X + X**2
@njit
def _moment_a(Y,X,theta):
    return _rho(X, 5/4) * (Y - (theta**2)*X - theta*(X**2))
@njit
def _moment_b(Y,X,theta):
    return _rho(X, theta) * (Y - (theta**2)*X - theta*(X**2))

def my_gmm2(Y,X, moment, init):
    """
    moment = 'a' or 'b'
    """
    n,_ = X.shape
    
    if moment == 'a':
        mom=_moment_a
    elif moment == 'b':
        mom=_moment_b
    else:
        print("moment should be 'a' or 'b'")
                                                  
    # objective function
    def obj(theta):
        # 1S
        g = np.mean(mom(Y,X, theta))
        J = lambda theta: n*(g**2)

        theta1 = optimize.fmin(J, init, disp=False)
        e = mom(Y,X,theta1)
        omega = np.dot((e-np.mean(e)).reshape(-1),(e-np.mean(e)).reshape(-1))/n  # centered
        
        # obj
        obj = n * g**2 / omega
        return obj
    coef = optimize.fmin(obj, init, disp=False)
    return coef[0]

In [10]:
Y,X = my_dgp(1000, 1)
my_gmm(Y,X,'b',-4)

-2.967675781249999

In [11]:
Y,X = my_dgp2(1000, 1)
my_gmm2(Y,X,'b',-4)

/tmp/ipykernel_30757/3258018131.py:15: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "my_dgp2" failed type inference due to: Cannot unify array(float64, 1d, C) and array(float64, 2d, C) for 'X.2', defined at /tmp/ipykernel_30757/3258018131.py (24)

File "../../../tmp/ipykernel_30757/3258018131.py", line 24:
<source missing, REPL/exec in use?>

During: typing of assignment at /tmp/ipykernel_30757/3258018131.py (24)

File "../../../tmp/ipykernel_30757/3258018131.py", line 24:
<source missing, REPL/exec in use?>

  @jit(fastmath=True)
/opt/conda/lib/python3.7/site-packages/numba/core/object_mode_passes.py:152: NumbaWarning: Function "my_dgp2" was compiled in object mode without forceobj=True.

File "../../../tmp/ipykernel_30757/3258018131.py", line 15:
<source missing, REPL/exec in use?>

  state.func_ir.loc))
/opt/conda/lib/python3.7/site-packages/numba/core/object_mode_passes.py:162: NumbaDeprecationWarning: 
Fall-back from the nopyt

-3.138574218749999

In [13]:
start = time.time()
for i in range(10):
    sys.stdout.write("\r {}/10".format(i+1))
    sys.stdout.flush()
    Y,X = my_dgp(1000,-1)
    my_gmm(Y,X,'b',-4)
end = time.time()
print("\nno numba: %s" % (end - start))

start = time.time()
for i in range(10):
    sys.stdout.write("\r {}/10".format(i+1))
    sys.stdout.flush()
    Y,X = my_dgp2(1000,-1)
    my_gmm2(Y,X,'b',-4)
end = time.time()
print("\nnumba: %s" % (end - start))

 10/10
no numba: 6.9173479080200195
 10/10
numba: 0.8824448585510254
